In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("data.csv")

In [3]:
data.shape

(569, 33)

Remove the first column 'id' and the last column 'Unnamed:32' from this dataset:

In [4]:
data.drop(labels=[data.columns[0],data.columns[32]],axis=1,inplace=True)

In [5]:
data.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


The task is to determine the best fitting probability distribution on each column data (Sample Data of the Random Variable) of the dataset, out of the possible choices of probability distributions: Normal(Symmetric), Rayleigh(Skewed) or Binomial. 

So, let's see that how are we going to do that for a single column data and then we have to repeat this process for rest of the columns: 

Step 1: First, we will check whether the Random Variable is Discrete or Continous by checking that how many unique values are there in the single column of the Random Variable and does the Random Variable has string or floating point or integer values? Usually, Discrete Random Variables have either Integer or String values whereas continous random variables have floating point values and also because the values are not sparse (0s or 1s) in continous random variables and they are dense (floating point values) so values don't repeat much and hence the number of unique values are almost equal to the total number of values in the column. As an example, lets take one by one two columns: 'diagnosis' and 'radius_mean': 

In [6]:
diagnosis_unique_values = data['diagnosis'].unique()

In [7]:
diagnosis_unique_values1 = data['compactness_worst'].unique()

In [8]:
len(diagnosis_unique_values1)

529

In [9]:
radius_mean_unique_values = data['radius_mean'].unique()

In [10]:
diagnosis_unique_values

array(['M', 'B'], dtype=object)

Let's see that in each of the cases, how many unique values are there:

In [11]:
len(diagnosis_unique_values)

2

In [12]:
len(radius_mean_unique_values)

456

As you can see that in our discrete random variable of 'diagnosis', we have very limited values, only two in number and they are string in nature : ['B, 'M']

Whereas, in our continous random variable of 'radius_mean', we can see the numbe rof unique values is huge and it's 456 which is not much smaller than 569, the number of rows in a column. Moreover, the nature or datatype of values is floating point. 

So, lets write this concept in the form of the code: 

In [13]:
def determine_random_variable_type(column_name):
    
    if (type(data[column_name]) == str or type(data[column_name]) == int) and (len(data[column_name].unique()) < len(data[column_name])):
        
        return 'discrete'
    
    else:
        
        return 'continous'

The above function will take the name of the single column of the dataset and access the data of that column from the dataset and check the datatype or nature of the values inside that column and also the total number of unique values in that column and check the conditions for the random variable or that specific column being discrete or continous and returns the the type of random variable as strings 'discrete' or 'continous'. 

Now, the next task is to calculate the value of the best estimator of the population parameter of the probability distribution from which a specific column data is being sampled. In case of discrete random variables, it's pretty easy to determine that: If there are only two unique values in the column data, then it's binomial distribution and multinomial if there are more than two unique values. 

But, the real challenge lies in case of continous random variables, where it is difficult to determine that from which continous probability distibution the column data is being sampled. We have two options here: Either Normal (Symmetric) or Rayleigh (Skewed). So, in this case we have to find the best fit of our data on both the distributions and then select the distribution with the BESTEST FIT. 

Well, now the question arrises that how we are going to do that. We know that the Log Likelihood Function Value calculated on the data determines the quality of fit of data on the presumed distribution and if we maximize that value then we get the best fit of a specific probability distribution on the data. so, we first find the best fit of normal distibution on our continous data and then we find the best fit of Rayleigh Distribution on our continous data by maximizing the Log Likelihood Function Value for both the cases and because we have to select that distribution which gives us the bestest fit so we will end up selecting the distribution whose Log Likelihood Function value is maximum among both maxmized Log Likelihood Function Values. 

The problem is that in order to find the Log Likelihood Function value evaluated in case of fit of specific distribution on the data, we need the PDF of that distribution, which we know but we dont know the values of best estimators of population parameters:
\begin{equation}
\mu, \sigma
\end{equation}
of Normal Distribution and best estimator of population parameter, 
\begin{equation}
\sigma (mode)
\end{equation}

of Rayleigh Distribution. 

Now, how do we get the values of the best estimators of population parameters of respective distributions. Well, we know the answer to this question and that is by Maximum Log Likelihood Estimation. We know that the data in each column of this dataset is IID (Independent and Identically Distributed) therefore, for any random column of this dataset, X where X can be 'radius_mean' or 'testure_mean' or anything else, the Log Likelihood function is evaluated as: 

\begin{equation}
\log_e L(\mu,\sigma) = \log_e P(X=x_1\cap X=x_2\cap .........X=x_N)
\end{equation}
For Normal Distribution

\begin{equation}
\log_e L(\mu, \sigma) = \log_e\prod\limits_{i=0}^{N}\left(\frac{1}{\sqrt{2\pi}\sigma}e^\frac{(x_i-\mu)^2}{2\sigma^2}\right)
\end{equation}

\begin{equation}
\log_e L(\mu, \sigma) = \sum\limits_{i=0}^{N}\log_e\left(\frac{1}{\sqrt{2\pi}\sigma}e^\frac{(x_i-\mu)^2}{2\sigma^2}\right)
\end{equation}
Here, the population mean and population standard deviation estimator best estimate for normal distribution can be evaluated by taking the derivative of the above function with respect to $\mu$ and $\sigma$ and when we do that, that is:
\begin{equation}
\frac{\partial\log_e L(\mu, \sigma)}{\partial\mu} = 0
\end{equation}

\begin{equation}
\frac{\partial log_e L(\mu, \sigma)}{\partial\sigma} = 0
\end{equation}

We get,
\begin{equation}
\mu_\text{best,normal} = \frac{\sum\limits_{i=0}^{N}{X_i}}{N}
\end{equation}

And, 

\begin{equation}
\sigma_\text{best,normal} = \sqrt{\frac{\sum\limits_{i=0}^{N}{(x_i-\mu_\text{best,normal})^2}}{N}}
\end{equation}

Which is sample mean and sample standard deviation, therefore these are the best estimates of the population parameters of Normal Distribution for the best fit on our column data. 

Similarly, we can find the values of the best estimators of the population parameter of Rayliegh Distribution by following the above mentioned steps, and we get the following value for the best estimate in case of Rayleigh Distribution:

\begin{equation}
\sigma_\text{best,rayleigh} = \sqrt{\frac{\sum\limits_{i=0}^{N}{x_i^2}}{2N}}
\end{equation}

So, these best estimates of the MVU Estimators of respective population parameters of respective Probability Distributions will maximize the Log Likelihood Function values for respective probability distributions and hence will result in best fit of respective distributions. Now, to determine the bestest fit among both of the distributions, we have to select that distribution for which the Log Likelihood Function is the highest among both already maximized values. 

Now, in our case N = 569, so the best estimates of the MVU estimators of the population parameters of respective probability distributions are given by:

\begin{equation}
\mu_\text{best,normal} = \frac{\sum\limits_{i=0}^{569}{x_i}}{569}
\end{equation}

And, 

\begin{equation}
\sigma_\text{best,normal} = \sqrt{\frac{\sum\limits_{i=0}^{569}{(x_i-\mu_\text{best,normal})^2}}{569}}
\end{equation}

For Normal Distribution

And, 

\begin{equation}
\sigma_\text{best,rayleigh} = \sqrt{\frac{\sum\limits_{i=0}^{569}{x_i^2}}{2*569}}
\end{equation}


Therefore, the maximized values of Log Likelihood Function for the case of both the probability distributions is given by:

\begin{equation}
L_\text{max,normal} = \log_e L(\mu_\text{best,normal},\sigma_\text{best,normal})
\end{equation}

\begin{equation}
L_\text{max,normal} = \sum\limits_{i=0}^{569}\log_e\left(\frac{1}{\sqrt{2\pi}\sigma_\text{best,normal}}e^\frac{(x_i-\mu_\text{best,normal})^2}{2\sigma_\text{best,normal}^2}\right)
\end{equation}

For Normal Distribution

And,

Similarly, maximized value of Log Likelihood Function for Rayleigh Distribution can be calculated as we have just calculated for Normal Distribution. 
\begin{equation}
L_\text{max,rayleigh} = \log_e L(\sigma_\text{best,rayleigh})
\end{equation}

Now, the column data will be considered to be sampled from Normal Distribution if:

\begin{equation}
L_\text{max,normal} > L_\text{max,rayleigh}
\end{equation}

else:

column data will be considered to be sampled from Rayleigh Distribution.

Therefore, in order to do all that, we are going to create following set of functions:

In [14]:
import scipy.stats as s

In [15]:
import numpy as np

In [16]:
def calculate_L_max_normal(column_name):
        
    mu_best_normal = data[column_name].mean()

    sigma_best_normal = data[column_name].std()

    L_max_normal =np.sum(s.norm.logpdf(data[column_name],mu_best_normal,sigma_best_normal))
        
    return L_max_normal

In [17]:
calculate_L_max_normal('radius_mean')

-1523.5944359203895

In [18]:
def calculate_L_max_rayleigh(column_name):
    
    sigma_best_rayleigh =np.sqrt(np.mean (data[column_name]**2)/2)
    
    L_max_rayleigh =np.sum(s.rayleigh.logpdf(data[column_name],scale=sigma_best_rayleigh))
    
    return L_max_rayleigh

In [19]:
calculate_L_max_rayleigh('radius_mean')

-1732.1508845394817

In [20]:
data

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [21]:
def determine_distribution_type(column_name):
    
    if determine_random_variable_type(column_name) == 'discrete':
        
        if len(data[column_name].unique()) == 2:
            
            return 'binomial'
        
        if len(data[column_name].unique()) > 2 or type(data[column_name]) == str or type(data[column_name]) == int:
            
            return 'multinomial'
        
    else:
        
        L_max_normal = calculate_L_max_normal(column_name) 
        
        L_max_rayleigh = calculate_L_max_rayleigh(column_name) 
        
        if L_max_normal > L_max_rayleigh:
            
            return 'normal'
        
        else:
            
            return 'rayleigh'

In [22]:
determine_distribution_type('radius_mean')

'normal'